In [1]:
# imports 

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import collections

np.random.seed(0)

2022-12-19 01:49:53.538163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 01:49:54.152259: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-19 01:49:55.462414: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-12-19 01:49:55.462554: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [2]:
# download MNIST data 

train, test = tff.simulation.datasets.emnist.load_data()

2022-12-19 01:49:59.976973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 01:50:00.058503: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 01:50:00.058903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 01:50:00.061656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
NUM_CLIENTS = len(train.client_ids)
NUM_CLIENTS

3383

**From the train dataset, we construct a dataset to simulate the "only positive labels" setting. This constitutes the following:**

1. Overall, we have a multi-label classification scheme. 

2. However, each local client only has access to labels from one class. 

3. In the original paper, the CIFAR-10 dataset is used for image classification in an "only positive labels" setting. 

4. However, we note that CIFAR-10 is a dataset which is not as reflective of how the "only positive labels" scheme could happen in real life as MNIST. This is because CIFAR-10 is originally sourced from the Tiny Images dataset, which was constructed via downloading images returned from internet search. Hence, there is no "personalization" aspect. On the other hand, MNIST is by nature and origin personalized -- for the total dataset of 60,000 images, there are 3383 total clients (or original writers) who wrote the handwritten digit. Hence, it is more realistic.

**As such, we separate train into 3383 different datasets which will then be distributed into 3383 participating clients in the federated learning process.**

**From the train dataset, we construct a dataset to simulate the "only positive labels" setting. This constitutes the following:**

1. Overall, we have a multi-label classification scheme. 

2. However, each local client only has access to labels from one class. 

3. In the original paper, the CIFAR-10 dataset is used for image classification in an "only positive labels" setting. 

4. However, we note that CIFAR-10 is a dataset which is not as reflective of how the "only positive labels" scheme could happen in real life as MNIST. This is because CIFAR-10 is originally sourced from the Tiny Images dataset, which was constructed via downloading images returned from internet search. Hence, there is no "personalization" aspect. On the other hand, MNIST is by nature and origin personalized -- for the total dataset of 60,000 images, there are 3383 total clients (or original writers) who wrote the handwritten digit. Hence, it is more realistic.

**As such, we separate train into 3383 different datasets which will then be distributed into 3383 participating clients in the federated learning process.**

In [ ]:
# obtain the most frequent label for each of the 3383 clients

import time

# d = {} # :400
# d2 = {} # 400:800
# d3 = {} # 800:1200
# d4 = {} # 1200:1600
# d5 = {} # 1600:2000
# d6 = {} # 2000:2400
# d7 = {} # 2400:2800
d8 = {} # 2800:NUM_CLIENTS
start = time.time()
for i in range(2800, NUM_CLIENTS):
    client_dataset = train.create_tf_dataset_for_client(train.client_ids[i])
    d8[i] = collections.defaultdict(int)
    for ex in client_dataset: 
        lab = ex['label'].numpy()
        d8[i][lab] += 1
end = time.time()

# combine the dictionaries together 

d1 = d
d1.update(d2)
d1.update(d3)
d1.update(d4)
d1.update(d5)
d1.update(d6)
d1.update(d7)
d1.update(d8)

In [ ]:
aux = dict(zip(d1.keys(), [max(d1[i], key=d1[i].get) for i in d1.keys()]))

import json
with open('most_frequent_labels.json', 'w') as fp:
    json.dump(aux, fp)